In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import plotly.express as px

# EDA


In [3]:
data_path = '/home/y/Documents/data/classify_leaves/'
trian_df = pd.read_csv(data_path + 'train.csv')
test_df = pd.read_csv(data_path + 'test.csv')
sub_df = pd.read_csv(data_path + 'sample_submission.csv')

In [4]:
trian_df.head()

,image,label
0,images/0.jpg,maclura_pomifera
1,images/1.jpg,maclura_pomifera
2,images/2.jpg,maclura_pomifera
3,images/3.jpg,maclura_pomifera
4,images/4.jpg,maclura_pomifera


In [5]:
test_df.head()

,image
0,images/18353.jpg
1,images/18354.jpg
2,images/18355.jpg
3,images/18356.jpg
4,images/18357.jpg


In [6]:
sub_df.head()

,image,label
0,images/18353.jpg,halesia_tetraptera
1,images/18354.jpg,robinia_pseudo-acacia
2,images/18355.jpg,chionanthus_virginicus
3,images/18356.jpg,ulmus_americana
4,images/18357.jpg,picea_pungens


In [7]:
# 看一下训练集中的类别
trian_df['label'].value_counts()

maclura_pomifera            353
ulmus_rubra                 235
prunus_virginiana           223
acer_rubrum                 217
broussonettia_papyrifera    214
                           ... 
ulmus_procera                58
cedrus_deodara               58
crataegus_crus-galli         54
evodia_daniellii             53
juniperus_virginiana         51
Name: label, Length: 176, dtype: int64

In [8]:
# 所有的图片都是224x224x3(长x宽x通道)
img_sampe1 = cv2.imread(data_path + 'images/1.jpg')
img_sampe2 = cv2.imread(data_path + 'images/2.jpg')
print(img_sampe1.shape)
print(img_sampe2.shape)

(224, 224, 3)
(224, 224, 3)
